In [1]:
import pandas as pd
from scipy import sparse

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(85271, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [3]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (554, 8753)
After:  (554, 1978)


In [5]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(5)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.066380,-0.020983,0.123590,0.019585,0.120745,0.202073,0.053347,-0.023833,-0.024468,...,0.026630,0.041215,0.152397,0.111276,0.054116,0.029598,0.016595,0.235932,0.082251,0.241162
(500) Days of Summer (2009),0.066380,1.000000,0.124532,0.285285,0.205735,0.170702,0.155897,0.176672,0.126614,0.168876,...,0.362892,0.183065,0.066874,0.400835,0.341237,0.264195,0.222215,0.058776,0.239501,0.129772
10 Cloverfield Lane (2016),-0.020983,0.124532,1.000000,0.008582,0.062864,0.019311,-0.010247,0.011265,-0.022009,0.264416,...,0.213361,0.114716,-0.020469,0.254265,0.257213,0.140910,0.308290,0.159607,0.072050,0.013131
10 Things I Hate About You (1999),0.123590,0.285285,0.008582,1.000000,0.280351,0.186191,0.213424,0.030867,0.043194,0.034800,...,0.279313,0.118879,0.070943,0.101343,0.148818,0.329511,0.078979,0.113045,0.159292,0.075588
"10,000 BC (2008)",0.019585,0.205735,0.062864,0.280351,1.000000,0.259174,0.128572,0.065598,-0.025744,0.107309,...,0.273020,0.088753,0.112637,0.199317,0.267550,0.233284,0.111211,0.055046,0.218045,0.096560


In [6]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [7]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

<ipython-input-7-068e9ff5f532>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
<ipython-input-7-068e9ff5f532>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
<ipython-input-7-068e9ff5f532>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
<ipython-input-7-068e9ff5f532>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),i

title,(500) Days of Summer (2009),Silver Linings Playbook (2012),Adventureland (2009),Up in the Air (2009),Toy Story 3 (2010),Alice in Wonderland (2010),"Notebook, The (2004)","Perks of Being a Wallflower, The (2012)",Kick-Ass (2010),Yes Man (2008),...,"Remains of the Day, The (1993)","Last Emperor, The (1987)",Stargate (1994),"Firm, The (1993)","Grifters, The (1990)","Madness of King George, The (1994)",Disclosure (1994),"Postman, The (Postino, Il) (1994)","English Patient, The (1996)",Clear and Present Danger (1994)
0,2.500000,1.223657,1.132359,1.129599,1.109408,1.089004,1.075190,1.073268,1.055686,1.044605,...,-0.125762,-0.127050,-0.132865,-0.132951,-0.136259,-0.150792,-0.154819,-0.161450,-0.171958,-0.222153
1,0.217801,0.219492,0.157540,0.244627,0.169051,0.500000,0.135728,0.122558,0.248208,0.148332,...,-0.002651,-0.020449,-0.004867,-0.013047,0.002434,-0.024271,-0.030808,-0.025986,-0.027678,-0.035450
2,-0.043631,-0.079664,-0.052940,-0.160892,-0.065792,-0.097927,-0.068949,-0.072585,-0.233549,0.012023,...,-0.050369,-0.180249,-0.308252,-0.027577,-0.291500,-0.136272,0.120526,-0.119584,-0.303484,-0.103561
3,-0.051749,-0.055612,-0.079471,-0.076672,-0.053582,-0.061607,-0.016323,-0.047387,-0.081509,-0.060543,...,-0.006678,-0.143900,-0.089972,-0.008434,-0.147700,-0.025367,0.027279,-0.057723,-0.113100,-0.018005


In [8]:
similar_movies.sum().sort_values(ascending=False).head(20)

title
(500) Days of Summer (2009)                      2.622421
Alice in Wonderland (2010)                       1.429471
Silver Linings Playbook (2012)                   1.307873
Help, The (2011)                                 1.161332
Toy Story 3 (2010)                               1.159086
Adventureland (2009)                             1.157488
Yes Man (2008)                                   1.144417
Up in the Air (2009)                             1.136662
Notebook, The (2004)                             1.125646
Crazy, Stupid, Love. (2011)                      1.107833
Perks of Being a Wallflower, The (2012)          1.075854
Holiday, The (2006)                              1.073698
Easy A (2010)                                    1.068188
Harry Potter and the Half-Blood Prince (2009)    1.067560
Marley & Me (2008)                               1.048526
About Time (2013)                                1.044138
50/50 (2011)                                     1.027541
Bruce Al

In [9]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

<ipython-input-9-e47dbbdb3052>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
<ipython-input-9-e47dbbdb3052>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
<ipython-input-9-e47dbbdb3052>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
<ipython-input-9-e47dbbdb3052>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),i

title
Amazing Spider-Man, The (2012)                           3.226301
Mission: Impossible III (2006)                           2.852170
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    2.717524
Over the Hedge (2006)                                    2.326136
Mission: Impossible - Ghost Protocol (2011)              2.207253
Fantastic Four (2005)                                    2.175030
Snakes on a Plane (2006)                                 2.174631
Chronicles of Riddick, The (2004)                        2.161321
Jumper (2008)                                            2.151958
Hancock (2008)                                           2.131789
Get Smart (2008)                                         2.105955
Crank (2006)                                             2.098183
Captain America: The First Avenger (2011)                2.093554
Die Another Day (2002)                                   2.078014
S.W.A.T. (2003)                                          2.063548
Resi